In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import time,csv

auth_provider = PlainTextAuthProvider(username='myuser', password='mypassword')

cluster = Cluster(["localhost"],port=9042,auth_provider=auth_provider, protocol_version=4)
session = cluster.connect('myschema')

currentToken = -9223372036854775808
maxToken = currentToken
maxTokenReached = False

i=0
with open('employee.csv',mode='a') as employee:
    employee_writer = csv.writer(employee,delimiter=',',quotechar='"')
    employee_writer.writerow(['emp_id','emp_name','emp_department_id'])
    while maxTokenReached==False:
        query = "select emp_id,emp_name,  emp_department_id, token(emp_id) from employee where token(emp_id) > %s limit 100" % currentToken
        print(i,query)
        print("\n")
        rows = session.execute(query)
        if rows[0] is None:
            maxTokenReached = True
        else:
            for row in rows:
                i +=1
                employee_writer.writerow([row.emp_id,row.emp_name,row.emp_department_id])
                currentToken = row.system_token_emp_id
        if currentToken > maxToken:
            maxToken=currentToken
        time.sleep(1)